In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense, Flatten, Dropout
from sklearn.model_selection import train_test_split

In [2]:
# 1. 데이터 로드 및 전처리
# rms_x, rms_y, ..., vel_peak2peak_z 38개 피처
# 타임스텝마다 데이터가 있다고 가정

df = pd.read_csv('55285839-9b78-48d8-9f4e-573190ace016_data.csv') # 데이터 로드

In [3]:
df = df.groupby('time')[['rms_x', 'rms_y', 'rms_z', 'rms_xyz', 'vel_rms_x', 'vel_rms_y', 'vel_rms_z', 'vel_rms_xyz',
                      'skewness_x', 'skewness_y', 'skewness_z', 'vel_skewness_x', 'vel_skewness_y', 'vel_skewness_z',
                      'kurtosis_x', 'kurtosis_y', 'kurtosis_z', 'vel_kurtosis_x', 'vel_kurtosis_y', 'vel_kurtosis_z',
                      'crest_factor_x', 'crest_factor_y', 'crest_factor_z', 'vel_crest_factor_x', 'vel_crest_factor_y', 'vel_crest_factor_z',
                      'peak_x', 'peak_y', 'peak_z', 'vel_peak_x', 'vel_peak_y', 'vel_peak_z',
                      'peak2peak_x', 'peak2peak_y', 'peak2peak_z', 'vel_peak2peak_x', 'vel_peak2peak_y', 'vel_peak2peak_z', 'imbalance_health']].mean().reset_index()

In [4]:
# 1. 데이터 로드 및 전처리
# rms_x, rms_y, ..., vel_peak2peak_z 38개 피처
# 타임스텝마다 데이터가 있다고 가정


X = df.iloc[:, 1:39].values  # 피처만 추출
y = df.iloc[:, 39:40].values # imbalance
#y = np.where(np.isnan(y), 1, y) # nan을 0(비정상)으로
y = np.where(y != 1, 0, 1) 

# 피처 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [5]:
# 2. 훈련 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [6]:
print(len(X_train), len(y_train), len(X_test), len(y_test))

189 189 48 48


In [7]:
# from imblearn.over_sampling import SMOTE

# smote = SMOTE()
# X_train, y_train = smote.fit_resample(X_train, y_train)
# X_test, y_test = smote.fit_resample(X_test, y_test)

In [8]:
print(len(X_train), len(y_train), len(X_test), len(y_test))

189 189 48 48


In [9]:
# 3. 1D-CNN 모델 구축
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # 이진 분류

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
# 모델 훈련
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
6/6 [==============================] - 1s 31ms/step - loss: 0.5622 - accuracy: 0.7831 - val_loss: 0.5544 - val_accuracy: 0.7708
Epoch 2/20
6/6 [==============================] - 0s 5ms/step - loss: 0.4824 - accuracy: 0.8254 - val_loss: 0.5775 - val_accuracy: 0.7708
Epoch 3/20
6/6 [==============================] - 0s 5ms/step - loss: 0.4764 - accuracy: 0.8254 - val_loss: 0.5478 - val_accuracy: 0.7708
Epoch 4/20
6/6 [==============================] - 0s 6ms/step - loss: 0.4596 - accuracy: 0.8254 - val_loss: 0.5374 - val_accuracy: 0.7708
Epoch 5/20
6/6 [==============================] - 0s 5ms/step - loss: 0.4488 - accuracy: 0.8254 - val_loss: 0.5464 - val_accuracy: 0.7708
Epoch 6/20
6/6 [==============================] - 0s 5ms/step - loss: 0.4483 - accuracy: 0.8254 - val_loss: 0.5699 - val_accuracy: 0.7708
Epoch 7/20
6/6 [==============================] - 0s 5ms/step - loss: 0.4413 - accuracy: 0.8254 - val_loss: 0.5599 - val_accuracy: 0.7708
Epoch 8/20
6/6 [=================

In [11]:
# 훈련 과정의 손실 및 정확도 출력
print("\n훈련 과정의 최종 손실 및 정확도:")
final_loss, final_accuracy = model.evaluate(X_test, y_test)
print(f"테스트 손실: {final_loss:.4f}, 테스트 정확도: {final_accuracy:.4f}")


훈련 과정의 최종 손실 및 정확도:
2/2 [==============================] - 0s 3ms/step - loss: 0.5946 - accuracy: 0.7500
테스트 손실: 0.5946, 테스트 정확도: 0.7500


In [12]:
from sklearn.metrics import classification_report

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.7).astype(int)  # 0.8 이상의 확률을 정상(1)으로 변환

# 정확도, 정밀도, 재현율, F1 스코어 계산 및 출력
print("\n분류 성능 보고서:")
print(classification_report(y_test, y_pred_classes, target_names=["비정상", "정상"]))

2/2 [==============================] - 0s 2ms/step

분류 성능 보고서:
              precision    recall  f1-score   support

         비정상       0.20      0.09      0.13        11
          정상       0.77      0.89      0.82        37

    accuracy                           0.71        48
   macro avg       0.48      0.49      0.47        48
weighted avg       0.64      0.71      0.66        48

